In [ ]:
# import modules

import matplotlib.pyplot as plt
import scipy as sp
from scipy import signal

In [ ]:
# initialize example plot

df_emg.plot(x='X[s]', y='Upper Trapezius: EMG 1')

In [ ]:
# EMG mean contraction correction

emg_trap = df_emg.iloc[:, 0]
emg_trap_mean = df_emg['Upper Trapezius: EMG 1'].mean()
df_emg["Upper Trapezius: EMG 1"] -= emg_trap_mean

In [ ]:
# plot mean contraction corrected EMG series

df_emg.plot(x='X[s]', y='Upper Trapezius: EMG 1')

In [ ]:
# create bandpass filter for EMG

high = 20/(1000/2)
low = 450/(1000/2)
b, a = sp.signal.butter(4, [high,low], btype='bandpass')

emg_filtered = sp.signal.filtfilt(b, a, df_emg['Upper Trapezius: EMG 1'])

In [ ]:
# plot series with bandpass filter applied

plt.plot(df_emg['X[s]'], emg_filtered)

In [ ]:
# rectify filtered signal

emg_rectified = abs(emg_filtered)

In [ ]:
# plot rectified signal

plt.plot(df_emg['X[s]'], emg_rectified)

In [ ]:
# apply basic analog filtering to remove DC offset or any other artifacts

low_pass = 10
sfreq = 1000

low_pass = low_pass/(sfreq/2)
b2, a2 = sp.signal.butter(4, low_pass, btype='lowpass')
emg_envelope = sp.signal.filtfilt(b2, a2, emg_rectified)

In [ ]:
# plot signal after analog filtering

plt.plot(df_emg['X[s]'], emg_envelope)

In [ ]:
# change axes to better visualize changes in signal

plt.axis([0, 10, 0, 0.000150])
plt.plot(df_emg['X[s]'], emg_envelope)

In [ ]:
# complete filtering and rectification for every muscle group

df_emg['Upper Trapezius: EMG 1'] = emg_envelope
muscles = ['Upper Trapezius: EMG 1', 'Posterior deltoid: EMG 2', 'Anterior deltoid: EMG 3',
                'Biceps brachii: EMG 4', 'Triceps brachii: EMG 5', 'Pectoralis major: EMG 6', 
                'Brachioradialis: EMG 7']

for muscle in muscles:
    # mean correction
    emg_mean = df_emg[muscle].mean()
    df_emg[muscle] -= emg_mean
    
    #filtering
    high = 20/(1000/2)
    low = 450/(1000/2)
    b, a = sp.signal.butter(4, [high,low], btype='bandpass')
    emg_filtered = sp.signal.filtfilt(b, a, df_emg[muscle])
    
    #rectification
    emg_rectified = abs(emg_filtered)
    
    #enveloping
    low_pass = 10
    sfreq = 1000
    low_pass = low_pass/(sfreq/2)
    b2, a2 = sp.signal.butter(4, low_pass, btype='lowpass')
    emg_envelope = sp.signal.filtfilt(b2, a2, emg_rectified)
    
    #resetting df
    df_emg[muscle] = emg_envelope

In [ ]:
# this function process_EMG can be used to complete all the steps found in this notebook

def process_EMG(df, df_emg):
    #process
    muscles = ['Upper Trapezius: EMG 1', 'Posterior deltoid: EMG 2', 'Anterior deltoid: EMG 3',
                    'Biceps brachii: EMG 4', 'Triceps brachii: EMG 5', 'Pectoralis major: EMG 6', 
                    'Brachioradialis: EMG 7']
    for muscle in muscles:
        # mean correction
        emg_mean = df_emg[muscle].mean()
        df_emg[muscle] -= emg_mean

        #filtering
        high = 20/(1000/2)
        low = 450/(1000/2)
        b, a = sp.signal.butter(4, [high,low], btype='bandpass')
        emg_filtered = sp.signal.filtfilt(b, a, df_emg[muscle])

        #rectification
        emg_rectified = abs(emg_filtered)

        #enveloping
        low_pass = 10
        sfreq = 1000
        low_pass = low_pass/(sfreq/2)
        b2, a2 = sp.signal.butter(4, low_pass, btype='lowpass')
        emg_envelope = sp.signal.filtfilt(b2, a2, emg_rectified)

        #resetting df
        df_emg[muscle] = emg_envelope

In [ ]:
# example function call

process_EMG(df1, df_emg1)